In [1]:
from pathlib import Path

import altair
import polars as pl

from power_of_10 import PowerOf10


In [2]:
temp_path = Path().home() / "OneDrive" / "Documents" / "Temp"

In [3]:
po10 = PowerOf10()
athlete = po10.get_athlete_by_id(1114055)

In [51]:
all_performances_df = (
    pl.DataFrame(athlete.all_performances)
    .filter(pl.col("Event") == "parkrun")
    .filter(pl.col("Meeting").str.starts_with("Eglinton parkrun"))
    .fill_null("")
    .with_columns(pl.lit("00:").alias("prefix"))
    .with_columns(pl.concat_str(pl.col("prefix"), pl.col("Performance"))
        .alias("Performance")
    )
    .with_columns(pl.col("Performance").str.split(":"))
    .with_columns(pl.duration(hours=pl.col("Performance").list.get(0), minutes=pl.col("Performance").list.get(1), seconds=pl.col("Performance").list.get(2)).alias("Result").dt.total_seconds())
    .select(pl.exclude("prefix", "Event", "Indoor", "Position", "Performance", "Meeting"))
    # .sort("Name", "Event", "Year")
)
all_performances_df

Date,Result
datetime[μs],i64
2025-02-15 00:00:00,1260
2025-03-29 00:00:00,1341
2024-03-30 00:00:00,1369
2024-09-28 00:00:00,1400
2024-01-06 00:00:00,1428
…,…
2018-05-12 00:00:00,2584
2018-01-06 00:00:00,2622
2018-04-14 00:00:00,2786


In [ ]:
df = (all_performances_df
.filter(pl.col("Event")=="parkrun")
#.with_columns(pl.col("Result").str.to_time("%H:%M:%S"))
.with_columns(pl.col("Performance").str.split(":").alias("s"))
.with_columns(pl.duration(hours=pl.col("s").list.get(0), minutes=pl.col("s").list.get(1), seconds=pl.col("s").list.get(2)).alias("Result"))
.select(pl.exclude("Indoor", "Event","s"))
)
df

In [52]:
all_performances_df.plot.line(x="Date", y="Result").properties(width=1200, height=500, title="Eglinton Parkrun").configure_scale(zero=False)

alt.Chart(...)